# Gene Expressions for different types of tumor
<div style="text-align: center;">
    <img src="images/gene_protein_cancer.jpg" alt="image" width="300" height="200">
</div>
[image src: https://www.cancer.gov/about-cancer/causes-prevention/genetics]

This project aims to identify different gene expressions associated to 5 types of tumor : 
- BRCA (Breast Cancer): Family of Genes (BRCA1 and BRCA2) are known as tumor suppresors. But mutation in these genes cause cancer.
- KIRC (Kidney Renal Clear Cell Carcinoma): 
- COAD (Colon Adenocarcinoma)
- LUAD (Lung Adenocarcinoma)
- PRAD (Prostate Adenocarcinoma)

The dataset is sourced from https://archive.ics.uci.edu/dataset/401/gene+expression+cancer+rna+seq.
The original dataset is published at https://www.synapse.org/Synapse:syn300013/discussion/threadId=5455. The Gene names in the dataset are dummy names. The actual gene names are at https://www.ncbi.nlm.nih.gov/gene, per this discussion thread https://www.synapse.org/Synapse:syn300013/discussion/threadId=5455. 

In [49]:
import warnings

warnings.filterwarnings("ignore")

DATA_ANALYSIS_DIR = "data-analysis/"
MODELS_DIR = "models/"
DATA_DIR = "data/"

## Data Loading


In [3]:
from time import time
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
from numpy import ndarray
from pandas import DataFrame
from pandas import Series
from pandas.core.indexes.base import Index


from sklearn.model_selection import train_test_split

from src.utils.ArrayUtils import get_list_of_items_in_both_lists



In [4]:
# read the data from files
url_reconstructed = 'TCGA-PANCAN-HiSeq-801x20531/data.csv'

# url = 'https://drive.google.com/file/d/1VXyhDXpYT8G2Buhkc6kBjw93CLG1y1f0/view?usp=drive_link'
# # Use only the Id and reconstruct the URL
# url_reconstructed = 'https://drive.google.com/uc?id=' + url.split('/')[-2]

df = pd.read_csv(url_reconstructed)
tumor_df = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/labels.csv')
df

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.000000
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.000000
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.000000
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.000000
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,sample_796,0.0,1.865642,2.718197,7.350099,10.006003,0.0,6.764792,0.496922,0.0,...,6.088133,9.118313,10.004852,4.484415,9.614701,12.031267,9.813063,10.092770,8.819269,0.000000
797,sample_797,0.0,3.942955,4.453807,6.346597,10.056868,0.0,7.320331,0.000000,0.0,...,6.371876,9.623335,9.823921,6.555327,9.064002,11.633422,10.317266,8.745983,9.659081,0.000000
798,sample_798,0.0,3.249582,3.707492,8.185901,9.504082,0.0,7.536589,1.811101,0.0,...,5.719386,8.610704,10.485517,3.589763,9.350636,12.180944,10.681194,9.466711,4.677458,0.586693
799,sample_799,0.0,2.590339,2.787976,7.318624,9.987136,0.0,9.213464,0.000000,0.0,...,5.785237,8.605387,11.004677,4.745888,9.626383,11.198279,10.335513,10.400581,5.718751,0.000000


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Columns: 20532 entries, Unnamed: 0 to gene_20530
dtypes: float64(20531), object(1)
memory usage: 125.5+ MB


In [6]:
print(f"Dataframe before adding the class column: {df.info()}")
print(f"Total # of columns: {len(df.columns)}")
print("COlumns type", type(df.columns))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Columns: 20532 entries, Unnamed: 0 to gene_20530
dtypes: float64(20531), object(1)
memory usage: 125.5+ MB
Dataframe before adding the class column: None
Total # of columns: 20532
COlumns type <class 'pandas.core.indexes.base.Index'>


In [7]:
# Copy over the Class column into the main dataframe
df = df.rename(columns={'Unnamed: 0': 'sample'})
tumor_df = tumor_df.rename(columns={'Unnamed: 0': 'sample'})
df['Class'] = np.where( (df['sample'] == tumor_df['sample']), tumor_df['Class'], df['sample'])

# df['Class'] = tumor_df['Class']
# print(f"Total # of columns: {len(df.columns)}")

df.drop(columns=['sample'], axis=1, inplace=True)

df.head()


,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,Class
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0,PRAD
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0,LUAD
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0,PRAD
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0,PRAD
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0,BRCA


## Exploratory Data Analysis (EDA)
### Data Cleaning and PreProcessing

Data : 
- No NANs as stated on the data source page


In [8]:
df.isnull().any().any()

np.False_

In [9]:
# Find columns with 0 values
zero_cols = df.columns[(df == 0).all()]
print(f"# of columns with all 0s in them : {zero_cols}")
df = df.drop(columns=zero_cols, axis=1)
print(f"# of columns with all 0s in them : {df.columns[(df == 0).all()]}")

# of columns with all 0s in them : Index(['gene_5', 'gene_23', 'gene_4370', 'gene_4808', 'gene_4809', 'gene_4814',
       'gene_4816', 'gene_4817', 'gene_4831', 'gene_5288',
       ...
       'gene_18908', 'gene_18909', 'gene_18910', 'gene_18911', 'gene_18914',
       'gene_18915', 'gene_19450', 'gene_19451', 'gene_19452', 'gene_19671'],
      dtype='object', length=267)
# of columns with all 0s in them : Index([], dtype='object')


In [10]:
# Post cleanup, write the data to file
df.to_csv(DATA_ANALYSIS_DIR + "df_PostColumnCleanup.csv")


In [11]:
# Class analysis
class_unique_vals = df['Class'].unique()
assert(len(class_unique_vals) == 5)

# print the distribution
print(f"Distribution of Class values: \n{df['Class'].value_counts()}")


Distribution of Class values: 
Class
BRCA    300
KIRC    146
LUAD    141
PRAD    136
COAD     78
Name: count, dtype: int64


In [12]:
# Split -
X = df.drop(columns=['Class'], axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df['Class'])


#### Summary : Data Cleaning and PreProcessing : 
- 0 NaNs
- 267 columns with only 0s as values. Dropped all of these columns
- 5 target values aka classifications


### Univariate and Multivariate Analysis
Given the large # of variables (columns), charting each out against other can be a challenge. Hence printing key insights such as stats to file.

##### Statistical Description of Data

In [ ]:
from scipy import stats

df_desc = df.describe()
df_desc_t = df_desc.T
# NOTE: We use this data to generate random test data when testing from App.
df_desc_t.to_csv(DATA_ANALYSIS_DIR + "df_describe.csv")

df_desc_zscore = df_desc.apply(stats.zscore)
df_desc_zscore.T.to_csv(DATA_ANALYSIS_DIR + "rawdata_zscore.csv")
# df_desc['upperbound'] = df_desc['mean'] + 3*(df_desc['std'])
# df_desc['lowerbound'] = df_desc['mean'] - 3*(df_desc['std'])


##### Correlation
Commented out since it takes too long to execute. Not much insight could be obtained.

In [ ]:
# --- NOTE : This take quite some time to run 11+ mins
# corr_df = X.corr()
# corr_df.to_csv(DATA_ANALYSIS_DIR + 'features_corr_matrix.csv')


In [ ]:
# Are there any Features with Correlation value= NaN ?
# -- In total there are 267 cols with 0 values, since they are deleted, result is zero
# corr_nan_list = corr_df.columns[corr_df.isna().all()].tolist()
# print(corr_nan_list)
# corr_df.head()

[]


,gene_0,gene_1,gene_2,gene_3,gene_4,gene_6,gene_7,gene_8,gene_9,gene_10,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
gene_0,1.000000,0.024111,0.027117,-0.080269,-0.028969,0.051855,-0.053647,-0.024423,-0.012785,-0.015423,...,-0.021109,-0.064341,-0.013874,-0.008384,-0.036623,0.034314,-0.000368,-0.049142,0.041554,-0.027767
gene_1,0.024111,1.000000,0.533902,0.148348,-0.158555,0.202855,0.177154,0.023411,0.006425,-0.022650,...,0.169172,0.190145,0.313725,-0.007284,0.123178,-0.255093,0.303771,0.247959,-0.067474,0.034348
gene_2,0.027117,0.533902,1.000000,0.075988,-0.125376,0.334900,0.136246,0.013228,-0.045725,0.011893,...,-0.021067,0.036411,0.242255,0.018755,-0.054584,-0.160446,0.163181,0.066709,0.011765,0.075604
gene_3,-0.080269,0.148348,0.075988,1.000000,0.153666,-0.007905,-0.084841,-0.033180,0.002434,-0.030119,...,-0.186729,-0.014547,-0.011733,-0.242855,-0.240551,0.092787,-0.000379,0.016733,-0.135829,0.023476
gene_4,-0.028969,-0.158555,-0.125376,0.153666,1.000000,-0.044347,-0.156698,0.082124,0.098267,-0.042986,...,-0.499208,-0.167764,-0.283480,0.179406,-0.367551,0.011739,-0.434604,-0.108036,-0.007229,0.062752


##### Statistical Description per Class

In [14]:
# Gene values (mean, mode, min, max, range) per class
# ---- NOTE : Below can take about 30secs to run.
# (X['gene_0'] == 0).sum()

def describeEachClass():
    for i in range(len(class_unique_vals)) :
        cls = class_unique_vals[i]
        # print('i= ', i, 'class= ', cls)
        df[df['Class'] == cls].describe().T.to_csv(f'{DATA_ANALYSIS_DIR}{cls}_describe_output.csv')

describeEachClass()
# print(type(df[df['Class'] == 'BRCA'].describe()))

##### Utility Functions

In [15]:
# Data has columns where values are close to 100% quartile. Get a list of these columns

def get_features_with_4thquartile_values_only(df) :
    '''
    @param df : Dataframe with all the data
    @return Dataframe consisting of descriptive stats for features in  in `df` where all values fall in 4th quartile.
    '''
    #generate descriptive statistics of given dataframe
    df_desc = df.describe()
    df_desc_t = df_desc.T

    print(f"Total # of columns: {len(df_desc_t)}")
    df_cols_outliers = df_desc_t[(df_desc_t['25%'] == 0) & (df_desc_t['50%'] == 0) & (df_desc_t['75%'] == 0)]
    print(f"# of columns with only outliers: {len(df_cols_outliers)}")

    return df_cols_outliers



In [16]:

# Identify if any of above columns have patterns such as associated with a type of tumor.
# Output the count of rows per Tumor type, for each of the above genes
def get_features_target_class_counts(df, df_cols_outliers, target_col='Class'):
    '''
    @param df : Dataframe with all the data
    @param df_cols_outliers : Dataframe with descriptive stats of features. See #get_features_with_4thquartile_values_only(df)
    @param target_col : Dependent variable in dataframe `df`.
    '''
    df_col_out_values = pd.DataFrame()
    for i in range(len(df_cols_outliers)):
        # for each record in outliers df, get only the rows that hv value greater than 0
        df_outliers_temp = df[df[df_cols_outliers.index[i]] > 0][[target_col, df_cols_outliers.index[i]]]
        df_outliers_temp1 = df_outliers_temp[target_col].value_counts().to_frame().T
        df_outliers_temp1['gene'] = df_cols_outliers.index[i]

        df_col_out_values = pd.concat([df_col_out_values, df_outliers_temp1], ignore_index=True)

        # df_col_out_values = pd.concat([df_col_out_values, df_outliers_temp], ignore_index=True)

    return df_col_out_values



##### Features with outliers

In [17]:
df_feature_outliers = get_features_with_4thquartile_values_only(df)
df_feature_outliers_count = get_features_target_class_counts(df, df_feature_outliers, 'Class')
df_feature_outliers_count.to_csv(f'{DATA_ANALYSIS_DIR}Outliers_cols_values_count.csv')


Total # of columns: 20264
# of columns with only outliers: 1660


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# init vars to help draw the layout
max_rows = 8
max_cols = 4
layout_diviser = min(max_rows, max_cols)

# vars to help place graph in the matrix
print(len(df_feature_outliers_count))
# no of features to plot per graph determined based on graph matrix size
features_per_graph = math.ceil( len(df_feature_outliers_count) / (max_rows * max_cols) )
start_index: int = 0
end_index: int = features_per_graph

# Total # of graphs to plot
total_graphs = math.ceil(len(df_feature_outliers_count) / features_per_graph)
print(f"# of feature groups: {total_graphs}")

# init plot
fig = make_subplots(rows=max_rows, cols=max_cols, shared_xaxes=True, vertical_spacing=0.03)

col_headers = df_feature_outliers_count.columns.to_list()

# Loop through # of graphs to plot. For each graph, plot the determined # of features
for fg in range(total_graphs):
    row = (fg) // layout_diviser
    col = (fg) % layout_diviser
    # print(f"row= {row} col= {col}")
    plot_df = df_feature_outliers_count.iloc[start_index:end_index, :]
    for i in range(len(plot_df)):
        fig.add_trace(go.Scatter(x=col_headers,
                                 y=plot_df.iloc[i, :-1],
                                 mode='lines',
                                 name=plot_df.loc[plot_df.index[i], 'gene']),
                                 row=row+1, col=col+1) #add_trace row and col, start index is 1, while row and col start with 0

    start_index = start_index + features_per_graph
    end_index = end_index + features_per_graph


fig.update_layout(height=1200, width=800, title_text="Outlier columns only : Records with (non-zero) values across the 5 classes")
fig.show()


1660
# of feature groups: 32


#### Summary : Univariate and Multivariate Analysis

- Descriptive Statistics of data : 1660 columns have mostly 0s, as values and their values fall into the 4th quartile.
- Outliers : Given the above 1660 columns / features have largely outliders, from above graph we identify the count of actual rows having these outlier values is less than 150, mostly less than 100. 
- Correlation : Given the large number of features, finding correlation between the features is challenging. The output file does not even open. Thus no inference to make.


### Feature Selection


### Dimensionality Reduction

#### PCA
To meet PCA needs, we need to ensure data confirms to below : 
- <u>correlation</u> between Independent vars.
- Variables must be <u>continuous</u> i.e numeric
- Variables must <u>be on same scale</u>. If one is in 1000s and another single digit, we will need to scale it.
- Data must be free from outliers

##### Utility functions

In [19]:
# get quartiles for each column
def get_quartiles(col):
    quartile1, quartile3 = col.quantile([0.25, 0.75])
    inter_quartile_range = quartile3 - quartile1
    lower_range = quartile1 - (1.5* inter_quartile_range)
    upper_range = quartile3 + (1.5* inter_quartile_range)

    return lower_range, upper_range

# lowerQ, upperQ = get_quartiles(X_train['gene_1'])
# print(f'lower quantile= {lowerQ}, upper quantile= {upperQ}')

In [20]:
# cols = X_train_pca.columns

def normalize_within_quartiles(df_X:DataFrame, cols:Index):
    '''
    @input df_X : Dataframe with ONLY independent variables.
    '''
    for i in cols :
        LL, UL = get_quartiles(df_X[i])
        # if (i == 'gene_0'):
        #     print(f"UL: {np.where(df_X[i] > UL, UL, df_X[i])}")
            # print(f"{np.where(df_X[i] < LL, LL, df_X[i]}")
        df_X[i] = np.where(df_X[i] > UL, UL, df_X[i])
        df_X[i] = np.where(df_X[i] < LL, LL, df_X[i])
    return df_X



In [21]:
# print(type(df_feature_outliers.index.to_list()))
def clean_input_data(X_dataframe: DataFrame, df_feature_outliers: DataFrame):
    # print(len(X_train.columns))
    X_train_pca = X_dataframe.drop(columns=df_feature_outliers.index.to_list())
    # X_train_pca.columns
    X_train_pca = normalize_within_quartiles(X_train_pca, X_train_pca.columns)
    X_train_pca.head(10)
    print(f"Total # of cols after removing quartiles: {len(X_train_pca.columns)}")

    return X_train_pca

##### Prep for PCA

In [22]:
# PCA -
# -- Is PCA the right approach to this data set where we are trying to identify Gene expressions for tumors.
# -- Shudnt we include every expression - small or big ?
from sklearn.decomposition import PCA

pca = PCA(random_state=42)


##### Remove Outliers
Based on data analysis on [outliers](#features-with-outliers) we know there are 1660 features with values in the 4th quartile. These gene expressions (features) are not specific to a tumor type (Class aka dependent variable) as observed in the output of function #get_features_target_class_counts. We will assume that these Gene expressions have not been recorded accurately or these have minimal influence from a tumor. Hence we remove these before running PCA.

In [23]:
X_train_pca = clean_input_data(X_train, df_feature_outliers)
X_train_pca.head(10)

Total # of cols after removing quartiles: 18604


,gene_1,gene_2,gene_3,gene_4,gene_6,gene_7,gene_10,gene_11,gene_12,gene_13,...,gene_20520,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529
391,4.094565,4.704501,7.545559,9.752299,8.595821,0.859970,1.152183,2.222650,2.222650,0.000000,...,9.532568,4.598568,7.654557,10.089887,7.107677,8.832156,12.816038,9.317229,9.461801,9.258929
148,6.237034,5.043235,6.297397,10.391415,7.669941,0.913033,0.527571,0.527571,2.752642,0.000000,...,8.136238,5.669299,8.437523,10.524503,1.467801,9.666187,11.258572,10.062033,9.018030,7.737159
595,3.028304,2.156138,6.459537,9.488378,5.319195,0.000000,1.043764,0.000000,3.703887,0.000000,...,7.647998,6.436851,8.557640,9.749657,5.406037,10.590297,11.449572,9.303614,9.228160,4.403943
770,2.706995,3.574586,6.090422,9.557619,7.162452,0.467175,0.467175,2.151112,2.667597,0.467175,...,9.467758,6.261474,9.005094,9.897212,6.405101,9.971630,12.057508,9.898369,9.835752,4.649144
493,3.522621,3.463021,8.133024,10.431341,7.536667,0.000000,0.000000,1.497740,1.901494,0.000000,...,10.234063,4.848788,8.581671,9.682631,0.542010,8.867365,12.051070,10.181786,8.857471,2.792439
706,3.913167,4.064193,6.530920,9.702678,8.404367,0.566084,1.547055,2.652119,1.547055,0.971663,...,9.741261,5.301701,8.260317,9.720046,5.513810,9.539921,12.177852,9.465385,9.314588,3.946366
526,2.463492,3.442293,6.850874,9.415991,7.270968,0.000000,0.485015,0.000000,3.458119,0.000000,...,7.722398,6.622088,9.022593,10.219907,2.886277,10.212630,11.027982,10.388082,9.968667,7.738680
480,2.676809,3.955629,6.639035,10.301165,7.946497,0.808426,0.460166,0.460166,3.933100,0.000000,...,7.855828,6.306275,9.071972,10.136145,7.163640,10.286477,10.573922,9.523879,10.355120,3.525868
223,2.870404,1.268614,6.516157,9.287569,6.674658,0.825541,0.000000,0.471031,1.109895,0.000000,...,8.173152,7.017599,8.560134,9.703589,5.989782,10.192428,11.646392,10.495325,9.885373,4.199633
455,3.069737,3.623200,6.744955,9.591219,7.254254,0.000000,2.204846,1.765450,3.268928,0.000000,...,7.722623,6.449447,8.276566,9.765221,3.339038,10.245957,12.200892,10.292529,9.378830,5.152070


#### Summary : Feature Selection
- Features with outliers values (in 4th quartile) dropped
- PCA reduces features to 640, from 18604 features. Data prepared for PCA execution in next section, as part of the Pipeline.


## Modeling

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import joblib



##### Utility Functions for Graphing and Analysis

In [25]:
# Function to plot Confusion Matrix
def plot_confusion_matrix(conf_matrix:ndarray, labels:ndarray, title="Confusion Matrix"):

    conf_matrix = conf_matrix[::-1]
    # labels = labels[::-1]

    #heat map
    fig = go.Figure(data=go.Heatmap(
        z=conf_matrix,
        x=labels,
        y=labels[::-1], #reverse the order to align labels with way Conf matrix is output
        colorscale='Rainbow', # 'Hot', # 'YlOrRd', # 'YlGnBu', #'Viridis',
        texttemplate="%{z}",
        textfont={"size": 10}
    ))

    fig.update_layout(
        title_text = title,
        xaxis_title="Predicted Class",
        yaxis_title="Actual Class",
        # xaxis={'side': 'top'},
        # yaxis={'autorange': 'reversed'},
        width=500,
        height=500,
    )

    fig.show()



In [26]:

# Using plotly's graph_objects
def plot_feature_importance_comparison_plotly(pca_model: PCA, classifier_model, feature_importances: Series, X_pre_pca_df: DataFrame):

    # top_features = feature_importances.head(10)

    # fig = px.bar(features_df, x=features_df.index, y=features_df[0])
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=feature_importances.index,
        y=feature_importances.values,
        marker=dict(color='indianred'),
        marker_color='indianred'
    ))
    fig.update_layout(title=f'Gene expression contribution to the model\n{classifier_model}', template='plotly_white')
    fig.show()



In [27]:
def get_pca_features_weights(pca_model: PCA, classifier_model, feature_names: Index) :
    '''
    Identify the overall weights of each feature in terms of its contribution to the given model. From the given Classifier model
    it retrieves feature importances and this data is merged with the PCA components matrix.
    @param pca_model : PCA object, after it has been fit / trained on the data
    @param classifier_model : Model used for classification
    @param feature_names : List of all features used, before PCA was run.
    @return Series consisting of contribution of each feature to the model
    '''

    # get feature contributions for each Principal Component
    # feature_names = X_pre_pca_df.columns
    n_components = len(pca_model.components_)
    pca_components_df = pd.DataFrame(pca_model.components_.T,
                                     columns=[f'PC{i+1}' for i in range(n_components)],
                                     index=feature_names)


    # Random forest importances for each component
    if (hasattr(classifier_model, 'feature_importances_')):
        rf_importances = pd.Series(
            classifier_model.feature_importances_,
            index=[f'PC{i+1}' for i in range(n_components)]
        )

        # --- calculate original feature importance by weighted combination ---
        orig_importances = pca_components_df.dot(rf_importances).abs() # we only care abt the magnitude
        # sum of all shud be 1, hence find each value's contrib to 100%
        orig_importances = orig_importances / orig_importances.sum()
        return orig_importances.sort_values(ascending=False)

    return None



In [28]:
def analyze_tree_path(pca_model: PCA, classifier_model, X_pca_df: DataFrame, y_series:Series, feature_names: Index, class_names=class_unique_vals):
    '''
    For each class / target, analyze the contribution of each feature by traversing the tree, finding the leaf node and
    then merging that with each feature's individual weights, calculated using the function #get_pca_features_weights.
    @param pca_model : PCA model, after the data has been fit aka trained on the data.
    @param classifier_model : Model used for classification, after it has been trained on the data
    @param X_pca_df : The DataFrame after PCA has been run and data has been transformed.
    @param y_series : The Y series corresponding to above X_pca_df dataset i.e if above is `test` dataset, this should also be test dataset
    @param feature_names : All the features used prior to running PCA
    @param class_names : List of unique Classes / Target Variables, the data represents. In this case we have the 5 tumors.
    @return DataFrame with columns as the 4 target classes and rows as features aka gene expressions.
    '''
    # feature_names = X_pca_df.columns
    #init dictionary that will hold each class details
    class_importances = {class_name: np.zeros(len(feature_names)) for class_name in class_unique_vals}

    #loop through each DecisionTree used by the model
    for tree in classifier_model.estimators_:
        tree_importances = tree.feature_importances_
        # get index of leaf node where sample is predicted
        leaf_nodes = tree.apply(X_pca_df)

        for row_idx, leaf_id in enumerate(leaf_nodes):
            # leaf_nodes has only the node number. Not the index that matches against corresponding index in y_series. Hence we use X_pca_df to get the index
            #row_idx is sequential increment of leaf_nodes
            record_index = X_pca_df.index[row_idx]
            predicted_class = y_series[record_index]

            original_importances = get_pca_features_weights(pca_model, classifier_model, feature_names)
            class_importances[predicted_class] += original_importances


    #Normalize
    for class_name in class_names:
        total = np.sum(class_importances[class_name])
        if total > 0:
            class_importances[class_name] /= total

    return pd.DataFrame(class_importances)



In [29]:
# Using SHAP to explain - #TODO
import shap

def plot_SHAP(pca_model:PCA, classifier_model, X_df: DataFrame):
    explainer = shap.Explainer(classifier_model)
    shap_values = explainer(X_df) #X_pca_dataframe
    original_shap = shap_values.values @ pca_model.components_


In [30]:
def convert_to_binary(y_series: Series):
    label_binzer = LabelBinarizer()
    label_binzer.fit(y_series)
    y_series_bin = np.array(label_binzer.transform(y_series))
    return y_series_bin


In [31]:
def create_pipeline(model):

    pca_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca),
        ('model', model)
    ])
    return pca_pipeline


def create_grid_model(model, model_params: dict):
    # call create Pipeline
    pipeline = create_pipeline(model)
    # cross validation param is default = 5. n_jobs configured as param
    grid_search = GridSearchCV(estimator=pipeline, param_grid=model_params, scoring='accuracy', n_jobs=1, refit=True, verbose=1)

    return grid_search


In [32]:
models_config = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'model__n_estimators':[50, 100, 200],
            'model__max_depth': [None, 10, 20],
            'model__min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        'params': {
            'model__n_estimators':[50, 100, 200],
            'model__max_depth': [None, 10, 20],
            'model__eta': [0.2, None, 0.4],
            #'model__n_jobs': [1]
        }
    }
}

#### Random Forest

In [33]:
# best_results = {}
def fit_model(model, model_params:dict, X_df: DataFrame, y_series:Series):
    '''
    @param model : Classifier model
    @param model_params : Params used for tuning the hyperparameters of the given model
    @param X_df : DataFrame of the independent vars with data
    @param y_series : A Series object with target class data.
    @return dict : Consisting of keys : Execution Time, Accuracy, Precision, Recall, F1-Score, Confusion matrix, Best Estimate, Best Params, Best Accuracy
    '''
    # for model_name, model_params in models_config.items():
    start_time = time()

    grid_search = create_grid_model(model=model, model_params=model_params)
    grid_search.fit(X_df, y_series)

    end_time = time()

    return grid_search



def predict_model(grid_search: GridSearchCV, X_df: DataFrame, y_series: Series):
    start_time = time()

    y_preds = grid_search.predict(X_df)

    end_time = time()

    best_results = {
        'Execution Time': (end_time - start_time),
        'Accuracy': accuracy_score(y_series, y_preds),
        'Precision': precision_score(y_series, y_preds, average='weighted'),
        'Recall': recall_score(y_series, y_preds,  average='weighted'),
        'F1-Score': f1_score(y_series, y_preds, average='weighted'),
        'Confusion matrix': confusion_matrix(y_series, y_preds),
        # 'GridSearchCV': grid_search,
        'Best Estimate': grid_search.best_estimator_,
        'Best Params': grid_search.best_params_,
        'Best Accuracy': grid_search.best_score_
    }

    return best_results



In [34]:

# best_results = fit_predict_model(X_train_pca, y_train)

# Random Forest :
rf_model = models_config['Random Forest']['model']
rf_model_config = models_config['Random Forest']['params']
rf_grid_search = fit_model(model=rf_model, model_params=rf_model_config, X_df=X_train_pca, y_series=y_train)



Fitting 5 folds for each of 18 candidates, totalling 90 fits


##### Model Evaluation

In [35]:
rf_X_test_pca = clean_input_data(X_test, df_feature_outliers)

rf_best_results = predict_model(grid_search=rf_grid_search, X_df=rf_X_test_pca, y_series=y_test)
print(rf_best_results)


Total # of cols after removing quartiles: 18604
{'Execution Time': 0.13870596885681152, 'Accuracy': 0.8819875776397516, 'Precision': 0.907548972766364, 'Recall': 0.8819875776397516, 'F1-Score': 0.8792621671637455, 'Confusion matrix': array([[60,  0,  0,  0,  0],
       [ 5, 11,  0,  0,  0],
       [ 2,  0, 28,  0,  0],
       [ 9,  0,  0, 18,  1],
       [ 2,  0,  0,  0, 25]]), 'Best Estimate': Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(random_state=42)),
                ('model', RandomForestClassifier(random_state=42))]), 'Best Params': {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 100}, 'Best Accuracy': np.float64(0.953125)}


In [36]:
rf_conf_matrix = rf_best_results['Confusion matrix']
plot_confusion_matrix(rf_conf_matrix, labels=class_unique_vals, title='Random Forest with PCA : Confusion Matrix')


In [37]:
# Estimator :
rf_pca_model = rf_grid_search.best_estimator_['pca']
rf_trained_model = rf_grid_search.best_estimator_['model']


In [40]:
# Find contribution of each feature to the model
rf_pca_weights_df = get_pca_features_weights(pca_model=rf_pca_model,
                                            classifier_model=rf_trained_model,
                                            feature_names=rf_X_test_pca.columns)
if rf_pca_weights_df is not None:
    # plot_feature_importance_comparison(pca_model, rd_model, pca_weights_df, X_pca_dataframe)
    plot_feature_importance_comparison_plotly(rf_pca_model, rf_model, rf_pca_weights_df, rf_X_test_pca)
else :
    print(f'PCA Weights for {rf_trained_model} could not be calculated')


In [48]:

rf_pca_weights_df[rf_pca_weights_df > 0][1:11]

gene_1362     0.000209
gene_69       0.000205
gene_16461    0.000194
gene_3894     0.000191
gene_11805    0.000190
gene_72       0.000184
gene_12053    0.000184
gene_6541     0.000184
gene_6126     0.000184
gene_19369    0.000184
dtype: float64

##### Persisting the model

In [54]:

joblib.dump(rf_trained_model, f"{MODELS_DIR}RForest_model.pkl")

['models/RForest_model.pkl']

In [44]:
# test to make sure persisted model can be reloaded correctly.
rf_model_joblib: RandomForestClassifier = joblib.load(f"{MODELS_DIR}RForest_model.pkl")
print(rf_model_joblib.feature_importances_)

[0.06421213 0.03487621 0.05939989 0.04714624 0.05313953 0.00856651
 0.01612737 0.01920058 0.00373367 0.00209029 0.00537764 0.0033241
 0.00185374 0.00349297 0.00856317 0.00218564 0.00341745 0.00194457
 0.00205509 0.00189443 0.00193965 0.00138935 0.00161562 0.00082214
 0.00158502 0.00104408 0.00123791 0.00125968 0.00114832 0.00145407
 0.00078465 0.00182096 0.00097389 0.00059082 0.00090904 0.00176744
 0.00081987 0.00077325 0.0012109  0.00104612 0.00149915 0.00109918
 0.00072159 0.00071322 0.00137605 0.00172924 0.00124583 0.00116851
 0.00067564 0.00065517 0.000756   0.00072681 0.00125472 0.00087051
 0.00112655 0.0005771  0.00114725 0.00129455 0.00072908 0.00104071
 0.0016147  0.00106499 0.00125455 0.00099827 0.00121893 0.00078208
 0.00102229 0.00107084 0.00082592 0.00049277 0.00075624 0.00109566
 0.00082247 0.00168365 0.00095885 0.00117501 0.00114096 0.00119474
 0.00101947 0.00084281 0.00171718 0.00063394 0.00124964 0.00048657
 0.00063976 0.00101306 0.00056967 0.00096836 0.00074156 0.00088

In [ ]:
#SHAP :
# plot_SHAP(pca_model=rf_pca_model, classifier_model=rf_trained_model, X_df=rf_X_test_pca)


#### XGBoost Model

In [53]:
# XGB requires target vars in binary format and hence cannot reuse the fn used for RandomForest
def fit_xgb_classifier(model: XGBClassifier, model_params:dict, X_df: DataFrame, y_series:ndarray):
    '''
    @param model : Classifier model
    @param model_params : Params used for tuning the hyperparameters of the given model
    @param X_df : DataFrame of the independent vars with data
    @param y_series : An ndarray object with binary representation of target classes
    @return dict : Consisting of keys : Execution Time, Accuracy, Precision, Recall, F1-Score, Confusion matrix, Best Estimate, Best Params, Best Accuracy
    '''
    # for model_name, model_params in models_config.items():
    start_time = time()

    grid_search = create_grid_model(model=model, model_params=model_params)
    grid_search.fit(X_df, y_series)

    end_time = time()

    return grid_search


def predict_xgb_classifier(grid_search: GridSearchCV, X_df: DataFrame, y_series_bin: ndarray):
    start_time = time()

    y_preds = grid_search.predict(X_df)

    end_time = time()

    y_test_bin_arr = np.array(y_series_bin)
    # confusion matrix
    cm = confusion_matrix(y_true=np.argmax(y_test_bin_arr, axis=1), y_pred=np.argmax(y_preds, axis=1)) #, labels=class_unique_vals)
    # plot_confusion_matrix(cm, labels=class_unique_vals, title='XGBoost with PCA : Confusion Matrix')

    best_results = {
        'Execution Time': (end_time - start_time),
        'Accuracy': accuracy_score(y_series_bin, y_preds),
        'Precision': precision_score(y_series_bin, y_preds, average='weighted'),
        'Recall': recall_score(y_series_bin, y_preds,  average='weighted'),
        'F1-Score': f1_score(y_series_bin, y_preds, average='weighted'),
        'Confusion matrix': cm,
        # 'GridSearchCV': grid_search,
        'Best Estimate': grid_search.best_estimator_,
        'Best Params': grid_search.best_params_,
        'Best Accuracy': grid_search.best_score_
    }

    return best_results



In [54]:
xgb_model = models_config['XGBoost']['model']
xgb_model_config = models_config['XGBoost']['params']

y_train_bin = convert_to_binary(y_train)
# print(y_train_bin)
# Uncomment below to identify mappings between binary format and the actual label
# 0=BRCA 1=COAD, 2=KIRC, 3=LUAD, 4=PRAD
# print("Binary data: ", y_train_bin)
# print("Label to int mapping: ", label_binzer.inverse_transform(np.array(y_train_bin)))

xgb_grid_search = fit_xgb_classifier(model=xgb_model, model_params=xgb_model_config, X_df=X_train_pca, y_series=y_train_bin)

# xgb_train_best_results = predict_xgb_classifier(grid_search=xgb_grid_search, X_df=X_train_pca, y_series_bin=y_train_bin)
# print(xgb_train_best_results)


Fitting 5 folds for each of 27 candidates, totalling 135 fits


##### Model Evaluation (XGB)

In [56]:
xgb_y_test_bin = convert_to_binary(y_test)
xgb_X_test_pca = clean_input_data(X_test, df_feature_outliers)

xgb_best_results = predict_xgb_classifier(grid_search=xgb_grid_search, X_df=xgb_X_test_pca, y_series_bin=xgb_y_test_bin)
print(xgb_best_results)


Total # of cols after removing quartiles: 18604
{'Execution Time': 0.13228607177734375, 'Accuracy': 0.9192546583850931, 'Precision': 0.9938906425007638, 'Recall': 0.9254658385093167, 'F1-Score': 0.9549868966430825, 'Confusion matrix': array([[60,  0,  0,  0,  0],
       [ 2, 14,  0,  0,  0],
       [ 1,  0, 29,  0,  0],
       [ 9,  0,  0, 19,  0],
       [ 1,  0,  0,  0, 26]]), 'Best Estimate': Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(random_state=42)),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eta=0.4,
                               eval_metric='mlogloss', feature_types=None,
                               feature_weights=None, gamma=None,
          

In [57]:
xgb_conf_matrix = xgb_best_results['Confusion matrix']
plot_confusion_matrix(xgb_conf_matrix, labels=class_unique_vals, title='XGBoost with PCA : Confusion Matrix')


In [58]:
xgb_pca_model = xgb_grid_search.best_estimator_['pca']
xgb_trained_model = xgb_grid_search.best_estimator_['model']


In [59]:
# Find contribution of each feature to the model
xgb_pca_weights_df = get_pca_features_weights(pca_model=xgb_pca_model,
                                            classifier_model=xgb_trained_model,
                                            feature_names=xgb_X_test_pca.columns)
if xgb_pca_weights_df is not None:
    # plot_feature_importance_comparison(pca_model, rd_model, pca_weights_df, X_pca_dataframe)
    plot_feature_importance_comparison_plotly(xgb_pca_model, xgb_trained_model, xgb_pca_weights_df, xgb_X_test_pca)
else :
    print(f'PCA Weights for {rf_trained_model} could not be calculated')


In [ ]:
# Persisting data for tests
xgb_X_test_pca.to_csv(DATA_ANALYSIS_DIR + 'xgb_X_test_pca.csv')

# Generate statistical description of these columns to help generate random values on the App side
len(xgb_X_test_pca.columns)
xgb_test_descr_df = xgb_X_test_pca.describe().T
xgb_test_descr_df.to_csv(DATA_DIR + 'xgb_test_X_describe.csv')


In [ ]:
xgb_pca_model.get_feature_names_out()

##### Persisting the Model

In [68]:
# Persist just the XGBooster model. If used the inputs would be 640 PCA components which can be identified using xgb_pca_model.get_feature_names_out()
joblib.dump(xgb_trained_model, f"{MODELS_DIR}XGBoost_model.pkl")

# persis the whole trained GridSearchCV
joblib.dump(xgb_grid_search, f"{MODELS_DIR}xgb_GridSearch_Pipeline.pkl")

['models/xgb_GridSearch_Pipeline.pkl']

In [69]:
# test to make sure persisted model can be reloaded correctly.
# xgb_model_joblib: XGBClassifier = joblib.load(f"{MODELS_DIR}XGBoost_model.pkl")
# print(xgb_model_joblib.feature_importances_)

xgb_gridcv_pipeline_joblib: GridSearchCV = joblib.load(f"{MODELS_DIR}xgb_GridSearch_Pipeline.pkl")
xgb_gridcv_pipeline_joblib.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(random_state=42)),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eta=0.4,
                               eval_metric='mlogloss', feature_types=None,
                               feature_weights=None, gamma=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=50, n_jobs=None, ...))])

In [ ]:
# Unit testing : to make sure model prediction is same as when input from Web App
unittest_data_X = pd.read_csv(DATA_ANALYSIS_DIR + 'request.csv')
# print("Printing req data: \n", type(unittest_data_X.iloc[0:1, 1:]))
unittest_data_pred = xgb_grid_search.predict(unittest_data_X.iloc[0:1, 1:])

# unittest_data_pred = xgb_grid_search.predict(xgb_X_test_pca.iloc[0:1, 0:])
unittest_data_pred

       gene_1    gene_2    gene_3     gene_4    gene_6    gene_7   gene_10  \
516  1.913914  3.568069  6.498854   9.865512  7.664440  1.830012  0.511873   
329  3.972858  3.368908  6.644179   9.361722  8.306344  0.431142  0.000000   
52   2.396708  2.399062  6.343800  11.105666  8.188876  0.000000  0.000000   
141  4.506799  4.120004  6.770155   9.845013  7.896054  0.000000  1.745539   
274  1.928276  2.447685  6.551608   9.822673  8.394810  1.620446  2.817597   
..        ...       ...       ...        ...       ...       ...       ...   
292  0.094249  2.713850  6.356100  10.335402  6.577912  0.000000  0.549324   
652  3.605542  3.712805  8.082452  10.125426  7.484662  1.397255  0.000000   
769  3.028056  2.514552  6.301622   9.497151  6.617021  0.700262  0.000000   
701  1.316435  1.872908  6.300018  10.289004  4.818983  0.547450  0.000000   
123  2.257765  3.514286  7.128232   9.710548  9.556673  0.985428  0.000000   

      gene_11   gene_12   gene_13  ...  gene_20520  gene_20521 

array([[0., 0., 1., 0., 0.]])

### Comparing the 2 Classifier Models

In [53]:
results_df = pd.DataFrame([rf_best_results, xgb_best_results]).T #.sort_values(by='Accuracy', ascending=False)
results_df.columns = ['Random Forest', 'XGBoost']
results_df

,Random Forest,XGBoost
Execution Time,0.232658,0.798206
Accuracy,0.881988,0.919255
Precision,0.907549,0.993891
Recall,0.881988,0.925466
F1-Score,0.879262,0.954987
Confusion matrix,"[[60, 0, 0, 0, 0], [5, 11, 0, 0, 0], [2, 0, 28...","[[60, 0, 0, 0, 0], [2, 14, 0, 0, 0], [1, 0, 29..."
Best Estimate,"(StandardScaler(), PCA(random_state=42), (Deci...","(StandardScaler(), PCA(random_state=42), XGBCl..."
Best Params,"{'model__max_depth': None, 'model__min_samples...","{'model__eta': 0.4, 'model__max_depth': None, ..."
Best Accuracy,0.953125,0.948438


### Code Under Construction
Please ignore below section as this code is being worked on

In [ ]:
# Area under Curve - REF : https://www.geeksforgeeks.org/interpreting-random-forest-classification-results/
# -------- #TODO : need to fix the roc_curve function
def plot_aoc_randomforest(rd_test_pred_proba: ndarray, y_test: Series) :
    target_vals = y_test.unique()
    # y_test_bin = label_binarize(y_test, classes=[0,1,2,3,4])
    label_binzer = LabelBinarizer()
    label_binzer.fit(y_test)
    y_test_bin = np.array(label_binzer.transform(y_test))
    print(y_test_bin[0])
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    print(f"y_test_bin : {y_test_bin[1]}")
    print(f"rd_preds_prob : {rd_test_pred_proba[:, 1]}")
    # print(f"y_test_bin : {len(y_test_bin[:, 1])}")
    # print(f"rd_preds_prob : {rd_preds_prob[:, 1]}")
    # print(rd_preds_prob)

    for index in range(len(target_vals)):
        fpr[index], tpr[index], _ = roc_curve(y_test_bin[index], rd_test_pred_proba[:, index])
        # print(f"FPR at {index}: \n{fpr[index]}")
        # print(f"TPR at {index}: \n{tpr[index]}")
        roc_auc[index] = auc(fpr[index], tpr[index])

    # Plot ROC curve
    # plt.figure()
    # for index in range(len(target_vals)) :
    #     plt.plot(fpr[index], tpr[index], lw=2, label=f"ROC curve of class {target_vals[index]} (area = {roc_auc[index]:.2f})")

    # # plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('Receiver Operating Characterstic for Tumor classes')
    # plt.legend(loc="lower right")
    # plt.show()

